# REGRESSION ANALYSIS

### IMPORT LIBRARIES

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from scipy import stats
from tqdm import tqdm

### IMPORT DATA

In [ ]:
states = ['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']

df_dict_state = {state:pd.read_csv('data_preparation_csv\%s.csv'%state, parse_dates = ['date']).set_index(['date'], drop = True) for state in states}

cluster_state = pd.Series({'AC': 1,
 'AL': 0,
 'AM': 1,
 'AP': 1,
 'BA': 0,
 'CE': 0,
 'DF': 0,
 'ES': 0,
 'GO': 0,
 'MA': 1,
 'MG': 0,
 'MS': 2,
 'MT': 0,
 'PA': 1,
 'PB': 0,
 'PE': 0,
 'PI': 0,
 'PR': 0,
 'RJ': 0,
 'RN': 0,
 'RO': 0,
 'RR': 1,
 'RS': 0,
 'SC': 0,
 'SE': 0,
 'SP': 2,
 'TO': 0}) #a14 24/3

### SHIFT LAG FUNCTION

In [ ]:
def shift(lag):
    for i,state in enumerate(states):
        iteration = df_dict_state[state][var_y + var_X + ['cluster']].copy().reset_index()
        iteration[var_X] = iteration[var_X].shift(lag)
        iteration['state'] = state
        iteration = iteration[iteration.date >= (pd.to_datetime('2021-01-16')) + pd.to_timedelta(lag,unit='D')]
        if i == 0:
            data = iteration
        else:
            data = pd.concat([data,iteration], ignore_index = True)


    for col in google_var:
        abs_max = np.abs(data[col]).max()
        data[col] = data[col] / abs_max

    for col in var_X:
        data[col] = 2 * ((data[col] - data[col].min())/(data[col].max() - data[col].min())) - 1

    return data

### INDENPENDENT VARIABLES CORRELATION MATRIX

In [ ]:
var_X = ['stringency_index_mm14',
       'retail_and_recreation_mm14', 'grocery_and_pharmacy_mm14',
       'parks_mm14', 'workplaces_mm14',
       'residential_mm14', 'hdi', 'demographic_density', 'doses_100k_mm14','cob_vacinal']

google_var = ['retail_and_recreation_mm14', 'grocery_and_pharmacy_mm14',
       'parks_mm14', 'workplaces_mm14',
       'residential_mm14']

for i,state in enumerate(states):
    iteration = df_dict_state[state].reset_index()
    iteration = iteration[iteration.date >= pd.to_datetime('2021-01-16')]
    if i == 0:
        data = iteration
    else:
        data = pd.concat([data,iteration], ignore_index = True)

correl = data[var_X].corr()
sns.heatmap(correl)

### DEFINE DEPENDENT VARIABLE

In [ ]:
var_y = ['deaths_100k_mm14']

### LAG ANALYSIS

In [ ]:
lag_scores = list()
for lag in tqdm(range(100)):
    for i,state in enumerate(states):
        iteration = df_dict_state[state][var_y + var_X + ['cluster']].copy().reset_index()
        iteration['state'] = state
        iteration[var_X] = iteration[var_X].shift(lag)
        iteration = iteration[iteration.date >= (pd.to_datetime('2021-01-16')) + pd.to_timedelta(lag,unit='D')]
        
        if i == 0:
            data = iteration
        else:
            data = pd.concat([data,iteration], ignore_index = True)
            
    data[var_X] = 2 * (data[var_X] - data[var_X].min())/(data[var_X].max() - data[var_X].min()) - 1
            
    X = data[var_X]
    y = data[var_y]
    
    est = sm.OLS(y, sm.add_constant(X)).fit()
    
    lag_scores.append(est.bic)
        

plt.plot(range(100),lag_scores)
plt.title('BIC X Lag')
plt.show()


### LAG DEFINTION

In [ ]:
lag = np.argmax(lag_scores)
lag

### BACKWISE FEATURE SELECTION FUNCTION

In [ ]:
def backwiseselection(X,y,selected_variables = X.columns,alfa = 0.05):
    #never fill selected_variables argument, it is used for recursion only#
    est = sm.OLS(y, sm.add_constant(X[selected_variables])).fit()
    p_values = est.pvalues.drop('const')
    if len(p_values[p_values < alfa]) == len(selected_variables):
        return selected_variables
    else:
        selected = selected_variables.drop(est.pvalues.drop('const').idxmax())
        return backwiseselection(X,y,selected,alfa)

### SELECTING VARIABLES

In [ ]:
X = shift(lag)[var_X]
y = shift(lag)[var_y]

for alfa in [0.001,0.01,0.05,0.1,1]:
    print("\n for alfa = %.3f, the eliminated features are: \n"%alfa)
    for feature in var_X:
        if feature not in backwiseselection(X,y,alfa = alfa):
            print(feature)

### LINEAR REGRESSION WITH SELECTED VARIABLES

In [ ]:
coefs = {cluster : pd.DataFrame(columns = ['LB','Betha','UB'], index = ['const'] + var_X, data = 0) for cluster in ['all',0,1,2]}

selected_X = X[backwiseselection(X,y,alfa = 0.05)]

est = sm.OLS(y, sm.add_constant(selected_X)).fit()

coefs['all'].loc[np.concatenate((['const'],np.array(selected_X.columns))),'Betha'] = est.params
coefs['all'].loc[np.concatenate((['const'],np.array(selected_X.columns))),'LB'] = est.conf_int(alpha = 0.05)[0]
coefs['all'].loc[np.concatenate((['const'],np.array(selected_X.columns))),'UB'] = est.conf_int(alpha = 0.05)[1]
coefs['all'].fillna(0,inplace=True)
   
print(est.summary())

###  LINEAR REGRESSION WITH SELECTED VARIABLES VISUALIZATION

In [ ]:
for i,x_i in enumerate(selected_X):
   plt.scatter(X[x_i],y)
   x_data = np.linspace(-1,1,2)
   plt.plot(x_data, est.params[i+1]*x_data + est.params[0], color = 'red')
   plt.title(var_y[0] + ' X %s, coef = %.4f'%(x_i,est.params[i+1]))
   plt.show()

### LINEAR REGRESSION WITH CLUSTERS

In [ ]:
ests = list()
for k in cluster_state.unique():
    
    data = shift(lag)
    X = data[data['cluster']==k][var_X]
    y = data[data['cluster']==k][var_y]

    print('\n for cluster \n', k)

    
    
    X_variables = backwiseselection(X,y,alfa = 0.05)

    selected_X = X[X_variables]
    y = y[var_y]
    
    print("removed variables for cluster k are:")
    for feature in var_X:
        if feature not in X_variables:
            print(feature)


    
    est = sm.OLS(y, sm.add_constant(selected_X)).fit()
    ests.append(est)
    
    
    coefs[k].loc[np.concatenate((['const'],np.array(X_variables))),'Betha'] = est.params
    coefs[k].loc[np.concatenate((['const'],np.array(X_variables))),'LB'] = est.conf_int(alpha = 0.05)[0]
    coefs[k].loc[np.concatenate((['const'],np.array(X_variables))),'UB'] = est.conf_int(alpha = 0.05)[1]
    coefs[k].fillna(0,inplace=True)

    print(est.summary())


###  LINEAR REGRESSION WITH SELECTED VARIABLES VISUALIZATION

In [ ]:
sse_all_vector = []
sse_cluster_vector = []
ratio_vector = []

for state in states:
    k = cluster_state[state]
    real_curve = data[data['state'] == state].set_index('date',drop=True)[var_y]
    parameters_values = data[data['state'] == state].set_index('date',drop=True)[var_X]
    estimated_curve_all = (coefs['all']['Betha'][1:] * parameters_values[var_X]).sum(axis=1) + coefs['all'].loc['const','Betha']
    estimated_curve_cluster = (coefs[k]['Betha'][1:] * parameters_values[var_X]).sum(axis=1) + coefs[k].loc['const','Betha']
    
    
    SSE_all = mean_squared_error(estimated_curve_all,real_curve)
    SSE_cluster = mean_squared_error(estimated_curve_cluster,real_curve)
    ratio = -100 * ((SSE_cluster/ SSE_all) - 1)
    
    sse_all_vector.append(SSE_all)
    sse_cluster_vector.append(SSE_cluster)
    ratio_vector.append(ratio)

    # Just a figure and one subplot
    f, ax = plt.subplots(figsize = (8,6))

    ax.set_title(state + " belonging to cluster %s"%k)
    ax.plot(real_curve, label="original y")
    ax.plot(estimated_curve_all, label="y_hat no cluster")
    ax.plot(estimated_curve_cluster, label="y_hat with cluster, SSE_improv = %.3f"%ratio)
    ax.legend()
    plt.savefig("graf_final%s.png"%state)
    plt.show()

### BETAS PLOT

In [ ]:
a = pd.DataFrame(columns = ['variable','estimated \u03B2','cluster'])

for k in coefs:
    iteration = coefs[k].reset_index().rename({'index':'variable'},axis=1)
    iteration = pd.concat([iteration[["variable","UB"]].rename({"UB":"estimated \u03B2"},axis=1),iteration[["variable","LB"]].rename({"LB":"estimated \u03B2"},axis=1)])
    iteration["cluster"] = k
    a = pd.concat([a,iteration])
a = a.reset_index(drop=True)

fig = plt.subplots(figsize=(15,12),dpi=350)
sns.set_theme(style="white")
ax = sns.pointplot(x="estimated \u03B2",y="variable",data=a,join = False,
                   hue = 'cluster',errwidth=0.8,capsize=0.2, dodge = 0.8)

for i in range(4):
    points = ax.collections[i]
    size = points.get_sizes().item()
    new_sizes = [size * 0.1 for name in ax.get_yticklabels()]
    points.set_sizes(new_sizes)
for i in range(12):
    ax.axhline((i-1/2),linewidth=0.5, color = 'grey')
ax.axvline(0,linewidth=0.5,color="red")

plt.show()

### EXPORT DATA

In [ ]:
## SSE Analysis
sse_all_vector.append(np.mean(sse_all_vector))
sse_cluster_vector.append(np.mean(sse_cluster_vector))
ratio_vector.append(np.mean(ratio_vector))
states = [' AC ', ' AL ', ' AM ', ' AP ', ' BA ', ' CE ', ' DF ', ' ES ', ' GO ', ' MA ', ' MG ', ' MS ', ' MT ', ' PA ', ' PB ', ' PE ', ' PI ', ' PR ', ' RJ ', ' RN ', ' RO ', ' RR ', ' RS ', ' SC ', ' SE ', ' SP ', ' TO ','MEAN']


## SSE Analysis 01
sse_all = pd.DataFrame(sse_all_vector).rename(columns = {0:'sse_all'})
sse_cluster = pd.DataFrame(sse_cluster_vector).rename(columns = {0:'sse_cluster'})
ratio = pd.DataFrame(ratio_vector).rename(columns = {0:'ratio'})
states = pd.DataFrame(states).rename(columns = {0:'state'})

sse_analysis = states.join(sse_all)
sse_analysis = sse_analysis.join(sse_cluster)
sse_analysis = sse_analysis.join(ratio)
sse_analysis.to_csv('sse_analysis.csv')


## SSE Analysis 02







